In [1]:
import numpy as np
%load_ext ipydex.displaytools

1. (4 Pts ) Propose a local search heuristic to solve the 0-1 Multi-Constraint Knapsack Problem (MKP). Ensure you deal with multiple, perhaps conflicting constraints, solution feasibility and the objective function.  How you balance and trade these off is the nature of your heuristic. Assume you are starting at a randomly generated feasible solution.

We will take a feasible solution knapsack and for each item already in the bag 
we will compare it to all of the items outside the sack to determine first if 
the switch will be an improvement, then if switching the two items is feasible.
This process will be repeated until a search has occured and no improvements have been found.

4. (2 Pts) Write up and comment pseudo code for your heuristic.

In [70]:
def knapsack_solver(item_attrs, item_utils, bag_cap, bag_contents, maximize=True):
    """
    item_attrs: Item Attributes, a (number of constraints) x (number of items)
        array containing each item's cost against bag constrains.

    item_vals: Item values, a 1 x (number of items) array containing the 
        values of each item's inclusion in the knapsack.

    bag_cap: Bag Capacity, a 1 x (number of constraints) array containing
        the value of the
    """
    num_items = np.shape(item_attrs)[1]

    # Begin loop until no change occurs
    change = True
    while change:
        change = False

        # Checking each of the current items
        for i in bag_contents.nonzero()[1]:
            this_item = item_attrs[:,i]
            this_util = item_utils[i]

            # Slack with this item removed
            slack = bag_cap - bag_contents @ item_attrs.T + this_item

            # Remove current contents from cancidates
            candidate_items = np.ones([1, num_items]) - bag_contents 

            # Remove less valuable items
            candidate_items =  candidate_items * np.greater(item_utils, this_util)
            
            # Remove items that wont fit
            candidate_items =  candidate_items * np.less(item_attrs, slack.T).all(axis=0)
            
            # Check if there is an improving item, then swap
            if bool(np.max(item_utils * candidate_items)):
                j = np.argmax(item_utils * candidate_items)
                bag_contents[0,i] = 0
                bag_contents[0,j] = 1
                change = True

        # Can another item fit in our slack space?
        slack = bag_cap - bag_contents @ item_attrs.T
        slack_eligible = np.less_equal(item_attrs, slack.T).all(axis=0)
        
        # If so, add it to the bag
        if slack_eligible.any():
            i = np.argmax(slack_eligible * item_utils)
            bag_contents[0,i] = 1
            change = True

    # End while
    # Return the new bag contents
    return bag_contents

2. (2 Pts) Create a small example (minimum of 3 constraints, 10 objects) of the 0-1 Multi-Constraint Knapsack Problem and apply your heuristic.

In [97]:
value = np.array([48, 30, 42, 36, 22, 43, 18, 24, 36, 29, 30, 25, 19, 41, 34, 32, 27, 24, 18])
attributes = np.array([
    [10, 30, 12, 22, 12, 20,  9,  9, 18, 20, 25, 18,  7, 16, 24, 21, 21, 32,  9], 
    [15, 20, 18, 20,  5, 12,  7,  7, 24, 30, 25, 20,  5, 25, 19, 24, 19, 14, 30], 
    [ 3,  1,  2,  3,  1,  2,  0,  2,  2,  1,  2,  3,  4,  3,  2,  3,  1,  1,  3]
])
capacity = np.array([[50, 50, 5]])
contents = np.array([[1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0]])

print(f"{'Items:':<20}{contents}")
print(f"{'Constraints:':<20}{contents @ attributes.T}")
print(f"{'Knapsack value:':<20}{contents @ value}")

result = knapsack_solver(attributes, value, capacity, contents)

print(f"{'Items:':<20}{result}")
print(f"{'Constraints:':<20}{result @ attributes.T}")
print(f"{'Knapsack value:':<20}{result @ value}")

Items:              [[1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0]]
Constraints:        [[28 29  5]]
Knapsack value:     [90]
Items:              [[1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]]
Constraints:        [[49 42  4]]
Knapsack value:     [96]


3. (2 Pts) Using the criteria found in Lecture 02, critique your heuristic. 

